In [ ]:
"""
Week 3: Multi-View Structure from Motion with Bundle Adjustment
CS436 Computer Vision - 3D Scene Reconstruction Project

Implements:
- Incremental PnP registration
- Map expansion with proper correspondence tracking
- Bundle adjustment for global refinement
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os
from glob import glob
from tqdm import tqdm
from scipy.optimize import least_squares
from scipy import stats
import plotly.graph_objects as go

# ============================================================================


# CONFIGURATION



In [ ]:
# CONFIGURATION
# ============================================================================

# For Google Colab - adjust path as needed
IMAGE_FOLDER = "/content/frames/ImagesForComputerVision"
MAX_DIM = 640  # Resize to manage memory
MIN_MATCHES_PNP = 8  # Minimum matches for PnP
MIN_MATCHES_TRIANGULATION = 15  # Minimum matches to triangulate new points
REPROJECTION_THRESHOLD = 8.0  # RANSAC threshold (pixels)

# ============================================================================


# STEP 1: LOAD AND RESIZE IMAGES



In [ ]:
# STEP 1: LOAD AND RESIZE IMAGES
# ============================================================================

print("="*70)
print("STEP 1: LOADING IMAGES")
print("="*70)

image_paths = sorted(glob(os.path.join(IMAGE_FOLDER, "*.*")))
images = []
for path in tqdm(image_paths, desc="Loading images"):
    img = cv2.imread(path)
    if img is None:
        continue
    h, w = img.shape[:2]
    scale = MAX_DIM / max(h, w)
    new_size = (int(w*scale), int(h*scale))
    resized = cv2.resize(img, new_size)
    images.append(resized)

print(f"✅ Loaded {len(images)} images. Size: {images[0].shape[:2]}")

# ============================================================================


# STEP 2: CAMERA INTRINSICS



In [ ]:
# STEP 2: CAMERA INTRINSICS
# ============================================================================

print("\n" + "="*70)
print("STEP 2: CAMERA INTRINSICS")
print("="*70)

h, w = images[0].shape[:2]

def get_camera_matrix(image_path, sensor_width_mm=9.8, default_fl_mm=6.9):
    """Extract focal length from EXIF or use default"""
    try:
        import piexif
        exif = piexif.load(image_path)
        fl = exif['Exif'][piexif.ExifIFD.FocalLength]
        fl_mm = fl[0] / fl[1]
        print(f"📷 EXIF focal length: {fl_mm:.2f} mm")
    except Exception:
        fl_mm = default_fl_mm
        print(f"📷 Using default focal length: {fl_mm:.2f} mm")
    
    fx = (fl_mm / sensor_width_mm) * w
    K = np.array([
        [fx, 0, w/2],
        [0, fx, h/2],
        [0, 0, 1]
    ], dtype=float)
    return K

K = get_camera_matrix(image_paths[0])
print(f"Camera matrix K:\n{K}")

# ============================================================================


# STEP 3: FEATURE EXTRACTION



In [ ]:
# STEP 3: FEATURE EXTRACTION
# ============================================================================

print("\n" + "="*70)
print("STEP 3: SIFT FEATURE EXTRACTION")
print("="*70)

sift = cv2.SIFT_create(nfeatures=2000)
keypoints_list = []
descriptors_list = []

for img in tqdm(images, desc="Extracting SIFT"):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    keypoints_list.append(kp)
    descriptors_list.append(des)

print(f"✅ Extracted features for {len(images)} images")

# ============================================================================


# STEP 4: FEATURE MATCHING



In [ ]:
# STEP 4: FEATURE MATCHING
# ============================================================================

print("\n" + "="*70)
print("STEP 4: FEATURE MATCHING (FLANN)")
print("="*70)

FLANN_INDEX_KDTREE = 1
flann = cv2.FlannBasedMatcher(
    dict(algorithm=FLANN_INDEX_KDTREE, trees=5),
    dict(checks=50)
)

def match_features(idx1, idx2, ratio_thresh=0.75):
    """Match features between two images using Lowe's ratio test"""
    des1, des2 = descriptors_list[idx1], descriptors_list[idx2]
    if des1 is None or des2 is None or len(des1) < 2 or len(des2) < 2:
        return []
    
    matches = flann.knnMatch(des1, des2, k=2)
    good = []
    for pair in matches:
        if len(pair) == 2:
            m, n = pair
            if m.distance < ratio_thresh * n.distance:
                good.append(m)
    return good

# ============================================================================


# STEP 5: TWO-VIEW INITIALIZATION



In [ ]:
# STEP 5: TWO-VIEW INITIALIZATION
# ============================================================================

print("\n" + "="*70)
print("STEP 5: TWO-VIEW INITIALIZATION")
print("="*70)

# Find good initial pair
i0, i1 = 0, 1
matches_01 = match_features(i0, i1)
print(f"Initial pair: images {i0} and {i1} with {len(matches_01)} matches")

# Extract point correspondences
pts1 = np.float32([keypoints_list[i0][m.queryIdx].pt for m in matches_01])
pts2 = np.float32([keypoints_list[i1][m.trainIdx].pt for m in matches_01])

# Estimate Essential Matrix and recover pose
E, mask_E = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, 
                                  prob=0.999, threshold=1.0)
_, R01, t01, mask_pose = cv2.recoverPose(E, pts1, pts2, K, mask=mask_E)

print(f"✅ Recovered initial pose with {mask_pose.sum()} inliers")

# Triangulate initial points
inliers = mask_pose.ravel().astype(bool)
pts1_inl = pts1[inliers]
pts2_inl = pts2[inliers]

P0 = K @ np.hstack((np.eye(3), np.zeros((3,1))))
P1 = K @ np.hstack((R01, t01))

X4 = cv2.triangulatePoints(P0, P1, pts1_inl.T, pts2_inl.T)
points_3d_hom = (X4[:3] / X4[3]).T

# Cheirality check - keep only points in front of both cameras
def check_positive_depth(P, points_3d):
    """Check if 3D points have positive depth"""
    points_h = np.hstack([points_3d, np.ones((points_3d.shape[0], 1))])
    points_cam = (P @ points_h.T).T
    return points_cam[:, 2] > 0

mask_cam0 = check_positive_depth(P0, points_3d_hom)
mask_cam1 = check_positive_depth(P1, points_3d_hom)
valid_mask = mask_cam0 & mask_cam1

points_3d = points_3d_hom[valid_mask]
pts1_valid = pts1_inl[valid_mask]
pts2_valid = pts2_inl[valid_mask]

print(f"✅ Initial reconstruction: {points_3d.shape[0]} 3D points")

# ============================================================================


# STEP 6: DATA STRUCTURES FOR INCREMENTAL SfM



In [ ]:
# STEP 6: DATA STRUCTURES FOR INCREMENTAL SfM
# ============================================================================

print("\n" + "="*70)
print("STEP 6: INITIALIZING SfM DATA STRUCTURES")
print("="*70)

# Store camera poses: camera_poses[img_idx] = (R, t)
camera_poses = {
    i0: (np.eye(3), np.zeros((3,1))),
    i1: (R01, t01)
}

# Store 3D points as a list (will grow dynamically)
point_cloud = [p for p in points_3d]

# Map: point_idx -> {img_idx: feature_idx}
# This tracks which 2D features correspond to each 3D point
point_observations = {}

# Build initial observations
matches_01_inliers = [matches_01[i] for i, valid in enumerate(inliers) if valid]
matches_01_valid = [matches_01_inliers[i] for i, valid in enumerate(valid_mask) if valid]

for pt_idx, match in enumerate(matches_01_valid):
    point_observations[pt_idx] = {
        i0: match.queryIdx,
        i1: match.trainIdx
    }

registered = {i0, i1}
print(f"✅ Initialized: {len(point_cloud)} points, {len(registered)} cameras")

# ============================================================================


# STEP 7: INCREMENTAL PnP REGISTRATION



In [ ]:
# STEP 7: INCREMENTAL PnP REGISTRATION
# ============================================================================

print("\n" + "="*70)
print("STEP 7: INCREMENTAL PNP REGISTRATION")
print("="*70)

def find_2d3d_correspondences(new_img_idx, registered_imgs):
    """
    Find 2D-3D correspondences for a new image.
    Returns: (points_3d, points_2d, point_indices)
    """
    pts_3d = []
    pts_2d = []
    pt_indices = []
    
    # Try matching with each registered image
    for reg_idx in registered_imgs:
        matches = match_features(reg_idx, new_img_idx)
        
        for m in matches:
            feat_idx_reg = m.queryIdx
            feat_idx_new = m.trainIdx
            
            # Check if this feature in registered image corresponds to a 3D point
            for pt_idx, obs in point_observations.items():
                if reg_idx in obs and obs[reg_idx] == feat_idx_reg:
                    pts_3d.append(point_cloud[pt_idx])
                    pts_2d.append(keypoints_list[new_img_idx][feat_idx_new].pt)
                    pt_indices.append((pt_idx, feat_idx_new))
                    break
    
    return np.array(pts_3d), np.array(pts_2d), pt_indices

# Register remaining images
unregistered = [i for i in range(len(images)) if i not in registered]

for new_idx in tqdm(unregistered, desc="Registering cameras"):
    # Find 2D-3D correspondences
    pts_3d, pts_2d, pt_indices = find_2d3d_correspondences(new_idx, list(registered))
    
    if len(pts_3d) < MIN_MATCHES_PNP:
        print(f"⚠️  Image {new_idx}: insufficient 2D-3D matches ({len(pts_3d)})")
        continue
    
    # Solve PnP with RANSAC
    success, rvec, tvec, inliers = cv2.solvePnPRansac(
        pts_3d, pts_2d, K, None,
        reprojectionError=REPROJECTION_THRESHOLD,
        confidence=0.99,
        flags=cv2.SOLVEPNP_ITERATIVE
    )
    
    if not success or inliers is None or len(inliers) < MIN_MATCHES_PNP:
        print(f"⚠️  Image {new_idx}: PnP failed")
        continue
    
    # Convert to rotation matrix
    R_new, _ = cv2.Rodrigues(rvec)
    t_new = tvec.reshape((3,1))
    
    # Store camera pose
    camera_poses[new_idx] = (R_new, t_new)
    registered.add(new_idx)
    
    # Update observations for inlier points
    for inlier_idx in inliers.ravel():
        pt_idx, feat_idx_new = pt_indices[inlier_idx]
        point_observations[pt_idx][new_idx] = feat_idx_new
    
    print(f"✅ Image {new_idx}: registered with {len(inliers)} inliers")

print(f"\n✅ Registered {len(registered)} / {len(images)} images")

# ============================================================================


# STEP 8: MAP EXPANSION - TRIANGULATE NEW POINTS



In [ ]:
# STEP 8: MAP EXPANSION - TRIANGULATE NEW POINTS
# ============================================================================

print("\n" + "="*70)
print("STEP 8: MAP EXPANSION")
print("="*70)

initial_point_count = len(point_cloud)

# For each pair of registered images, triangulate new points
registered_list = sorted(list(registered))

for i in tqdm(range(len(registered_list)), desc="Triangulating new points"):
    for j in range(i+1, len(registered_list)):
        idx1 = registered_list[i]
        idx2 = registered_list[j]
        
        # Match features
        matches = match_features(idx1, idx2)
        if len(matches) < MIN_MATCHES_TRIANGULATION:
            continue
        
        # Get camera poses
        R1, t1 = camera_poses[idx1]
        R2, t2 = camera_poses[idx2]
        P1 = K @ np.hstack((R1, t1))
        P2 = K @ np.hstack((R2, t2))
        
        # Extract matched points
        pts1_all = np.float32([keypoints_list[idx1][m.queryIdx].pt for m in matches])
        pts2_all = np.float32([keypoints_list[idx2][m.trainIdx].pt for m in matches])
        feat_idx1 = [m.queryIdx for m in matches]
        feat_idx2 = [m.trainIdx for m in matches]
        
        # Check which matches are NOT already triangulated
        new_matches_mask = []
        for k, (m, f1, f2) in enumerate(zip(matches, feat_idx1, feat_idx2)):
            already_exists = False
            for pt_idx, obs in point_observations.items():
                if idx1 in obs and obs[idx1] == f1:
                    already_exists = True
                    break
            new_matches_mask.append(not already_exists)
        
        new_matches_mask = np.array(new_matches_mask)
        if new_matches_mask.sum() < 4:
            continue
        
        pts1_new = pts1_all[new_matches_mask]
        pts2_new = pts2_all[new_matches_mask]
        feat_idx1_new = [feat_idx1[k] for k in range(len(feat_idx1)) if new_matches_mask[k]]
        feat_idx2_new = [feat_idx2[k] for k in range(len(feat_idx2)) if new_matches_mask[k]]
        
        # Triangulate
        X4 = cv2.triangulatePoints(P1, P2, pts1_new.T, pts2_new.T)
        new_points_3d = (X4[:3] / X4[3]).T
        
        # Cheirality check
        mask1 = check_positive_depth(P1, new_points_3d)
        mask2 = check_positive_depth(P2, new_points_3d)
        valid = mask1 & mask2
        
        # Add valid points to map
        for k in range(len(new_points_3d)):
            if valid[k]:
                pt_idx = len(point_cloud)
                point_cloud.append(new_points_3d[k])
                point_observations[pt_idx] = {
                    idx1: feat_idx1_new[k],
                    idx2: feat_idx2_new[k]
                }

new_points = len(point_cloud) - initial_point_count
print(f"✅ Added {new_points} new 3D points")
print(f"Total points in map: {len(point_cloud)}")

# ============================================================================


# STEP 9: BUNDLE ADJUSTMENT



In [ ]:
# STEP 9: BUNDLE ADJUSTMENT
# ============================================================================

print("\n" + "="*70)
print("STEP 9: BUNDLE ADJUSTMENT")
print("="*70)

def project_points(point_3d, R, t, K):
    """Project 3D point to 2D using camera parameters"""
    point_cam = R @ point_3d + t.ravel()
    point_2d_hom = K @ point_cam
    return point_2d_hom[:2] / point_2d_hom[2]

def residuals_ba(params, n_cameras, n_points, camera_indices, point_indices, points_2d, K):
    """
    Compute residuals for bundle adjustment.
    params: [camera_params (6*n_cameras), point_params (3*n_points)]
    """
    camera_params = params[:n_cameras * 6].reshape((n_cameras, 6))
    points_3d = params[n_cameras * 6:].reshape((n_points, 3))
    
    residuals = []
    
    for i in range(len(camera_indices)):
        cam_idx = camera_indices[i]
        pt_idx = point_indices[i]
        observed_2d = points_2d[i]
        
        # Get camera parameters (rvec, tvec)
        rvec = camera_params[cam_idx, :3]
        tvec = camera_params[cam_idx, 3:].reshape(3, 1)
        R, _ = cv2.Rodrigues(rvec)
        
        # Project point
        projected = project_points(points_3d[pt_idx], R, tvec, K)
        
        # Compute residual
        residuals.append(observed_2d[0] - projected[0])
        residuals.append(observed_2d[1] - projected[1])
    
    return np.array(residuals)

# Prepare data for bundle adjustment
camera_indices = []
point_indices = []
points_2d_obs = []

camera_id_map = {img_idx: i for i, img_idx in enumerate(sorted(registered))}
point_id_map = {i: i for i in range(len(point_cloud))}

for pt_idx, observations in point_observations.items():
    for img_idx, feat_idx in observations.items():
        if img_idx in camera_id_map:
            camera_indices.append(camera_id_map[img_idx])
            point_indices.append(point_id_map[pt_idx])
            points_2d_obs.append(keypoints_list[img_idx][feat_idx].pt)

camera_indices = np.array(camera_indices)
point_indices = np.array(point_indices)
points_2d_obs = np.array(points_2d_obs)

print(f"Bundle adjustment setup:")
print(f"  Cameras: {len(camera_id_map)}")
print(f"  Points: {len(point_cloud)}")
print(f"  Observations: {len(camera_indices)}")

# Initialize parameters
n_cameras = len(camera_id_map)
n_points = len(point_cloud)

camera_params_init = np.zeros((n_cameras, 6))
for img_idx, cam_id in camera_id_map.items():
    R, t = camera_poses[img_idx]
    rvec, _ = cv2.Rodrigues(R)
    camera_params_init[cam_id, :3] = rvec.ravel()
    camera_params_init[cam_id, 3:] = t.ravel()

points_3d_init = np.array(point_cloud)

# Concatenate all parameters
x0 = np.hstack([camera_params_init.ravel(), points_3d_init.ravel()])

print(f"\nRunning bundle adjustment optimization...")
print(f"Initial parameters: {len(x0)}")

# Run optimization
result = least_squares(
    residuals_ba,
    x0,
    args=(n_cameras, n_points, camera_indices, point_indices, points_2d_obs, K),
    verbose=2,
    max_nfev=100,  # Limit iterations for Colab
    ftol=1e-4,
    method='trf'
)

print(f"\n✅ Bundle adjustment complete!")
print(f"  Initial cost: {result.fun.sum():.2f}")
print(f"  Final cost: {np.sum(result.fun**2):.2f}")
print(f"  Iterations: {result.nfev}")

# Extract optimized parameters
camera_params_opt = result.x[:n_cameras * 6].reshape((n_cameras, 6))
points_3d_opt = result.x[n_cameras * 6:].reshape((n_points, 3))

# Update camera poses
for img_idx, cam_id in camera_id_map.items():
    rvec_opt = camera_params_opt[cam_id, :3]
    tvec_opt = camera_params_opt[cam_id, 3:]
    R_opt, _ = cv2.Rodrigues(rvec_opt)
    camera_poses[img_idx] = (R_opt, tvec_opt.reshape(3, 1))

# Update point cloud
point_cloud_refined = points_3d_opt

print(f"\n✅ Point cloud refined: {len(point_cloud_refined)} points")

# ============================================================================


# STEP 10: OUTLIER REMOVAL



In [ ]:
# STEP 10: OUTLIER REMOVAL
# ============================================================================

print("\n" + "="*70)
print("STEP 10: OUTLIER REMOVAL")
print("="*70)

# Remove statistical outliers
z_scores = np.abs(stats.zscore(point_cloud_refined, axis=0))
outlier_mask = (z_scores < 3).all(axis=1)
points_clean = point_cloud_refined[outlier_mask]

print(f"Points before outlier removal: {len(point_cloud_refined)}")
print(f"Points after outlier removal: {len(points_clean)}")
print(f"Removed {len(point_cloud_refined) - len(points_clean)} outliers")

# ============================================================================


# STEP 11: VISUALIZATION



In [ ]:
# STEP 11: VISUALIZATION
# ============================================================================

print("\n" + "="*70)
print("STEP 11: 3D VISUALIZATION")
print("="*70)

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_clean)
pcd.paint_uniform_color([0.2, 0.6, 1.0])
pcd.estimate_normals()

# Save to file
output_ply = "/content/week3_refined_sparse.ply"
o3d.io.write_point_cloud(output_ply, pcd)
print(f"✅ Saved refined point cloud: {output_ply}")

# Plotly visualization
fig = go.Figure(data=[go.Scatter3d(
    x=points_clean[:,0],
    y=points_clean[:,1],
    z=points_clean[:,2],
    mode='markers',
    marker=dict(
        size=2,
        color=points_clean[:,2],
        colorscale='Viridis',
        opacity=0.8,
        colorbar=dict(title="Depth (Z)")
    )
)])

fig.update_layout(
    title="Week 3: Refined Sparse Point Cloud (After Bundle Adjustment)",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    width=900,
    height=700
)

fig.show()

# ============================================================================


# STEP 12: FINAL SUMMARY



In [ ]:
# STEP 12: FINAL SUMMARY
# ============================================================================

print("\n" + "="*70)
print("WEEK 3 DELIVERABLE SUMMARY")
print("="*70)
print(f"\n✅ Input: {len(images)} images")
print(f"✅ Registered cameras: {len(registered)} / {len(images)}")
print(f"✅ Initial 3D points: {initial_point_count}")
print(f"✅ After map expansion: {len(point_cloud)}")
print(f"✅ After bundle adjustment & cleaning: {len(points_clean)}")
print(f"✅ Total observations: {len(camera_indices)}")
print(f"\n✅ Output saved to: {output_ply}")
print("="*70)